In [ ]:
#@title Imports

!pip install gensim
!pip install en_core_web_sm
!pip install networkx
!pip install scipy
!pip install pyvis
!pip install pyLDAvis

from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import networkx as nx
import nltk
import matplotlib.pyplot as plt
from pyvis.network import Network
from gensim.models.phrases import Phrases, Phraser
import json
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
from gensim.parsing.preprocessing import STOPWORDS
import json
import requests as rq
from bs4 import BeautifulSoup
import time
import spacy
nlp = spacy.load("en_core_web_sm")
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import unicodedata
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
#@title NLTK Downloads
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Get the AFINN dictionary from GitHub
lexicon = requests.get("https://raw.githubusercontent.com/MikoBie/ids/main/notebooks/AFINN-111.txt").text.strip().lower()
## Split the string by new light sign -- \n
lexicon = [ w.strip() for w in lexicon.split("\n") ]
## Convert a list of strings, where the word and value are separated by \t
## (tab), into a dictionary, where a word is a key and an integer is a value.
lexicon = { word: int(score) for word, score in map(lambda x: x.split("\t"), lexicon) }

## Download lexicons etc.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download('omw-1.4')

## Get stopwords for english and initialize lemmatizer
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [342]:
#@title LDA MyCorpus
class MyCorpus:
    """
    A class that represents a corpus and has usefull methods defined.

    """
    
    def __init__(self, path, key='content'):
        """
        Reads from a JSON line file. Tokenizes and lemmatizes
        the text under key. It writes out the new JSON line
        file with a new field -- tokens.
        Args:
            path (str): a path to a JSON line.
            key (str): a key with the content to lemmatize.
        """
        self._path_original = path
        self._key = key
        self._dictionary = None
        self._path = path.replace('.', '_NLP.')
        with open(self._path, 'w') as file:
            n = 1
            for line in open(self._path_original, 'r'):
                temp_dict = json.loads(line)
                text_nlp = nlp(temp_dict[self._key])
                temp_dict['tokens'] = []
                for token in text_nlp:
                    is_stop = token.is_stop or token.is_punct or token.is_space \
                        or token.is_bracket or token.is_currency or token.is_digit \
                            or token.is_quote or len(token) < 2
                    if is_stop:
                        continue
                    else:
                        temp_dict['tokens'].append(token.lemma_.lower())
                file.write( json.dumps(temp_dict) + '\n')
                sys.stdout.write(f'\rLine {n} processed')
                n += 1
                sys.stdout.flush()

        
    def set_dictionary(self, dictionary):
        """
        Assigns a gensim.corpora.dictionary.Dictioanry object
        to self._dictionary.

        Args:
            dictionary (gensim.corpora.dictionary.Dictionary): a dictionary
            that stores the frequencies of unique tokens in the corpus.
        """
        self._dictionary = dictionary

    def get_tokens(self):
        """
        It takes the path to a JSON line file with comments from Reddit and
        returns a generator that yields tokens for each comment.

        Yields:
            list : list of tokens for a comment from Reddit. 
        """
        for doc in open(self._path, 'r'):
            temp = json.loads(doc)
            yield temp['tokens']
    
    def get_bow(self):
        """
        It takes a dictionary with frequencies of unique tokens in the corpus
        and for each list of tokens returns a list of tuples that denote the 
        id of a given token and its frequency in a given document.

        Raises:
            ValueError: if the dictionary was not assigned to self._dictionary.

        Yields:
            list : a list of tuples that denote the id of a given token and its
            frequency in a given document.
        """
        if self._dictionary:
            for doc in self.get_tokens():
                yield self._dictionary.doc2bow(doc)
        else:
            raise ValueError('Dictionary has the value of None')
    
    def __iter__(self):
        """
        Yields:
            list : a list of tuples that denote the id of a given token and
            its frequency in a given document.
        """
        for doc in self.get_bow():
            yield doc

    def get_topics(self, model):
        """
        It takes a model and returns a generator that yields a mapping for each
        comment from Reddit. Among other keys it returns the most probable topic
        based on the LDA model provided and its probability.

        Args:
            model (gensim.models.ldamodel.LdaModel): Latent Dirchlet Allocation
            model.

        Yields:
            dict : a mapping for each comment from Reddit. Among other keys it
            returns the most prpobable topic based on the LDA model provided and
            its probability. 
        """
        for doc in open(self._path, 'r'):
            temp = json.loads(doc)
            topics = model.get_document_topics(self._dictionary.doc2bow(temp['tokens']))
            topic, prob = sorted( topics, key = lambda x: x[1], reverse=True )[0]
            temp['topic'] = topic + 1
            temp['topic_prob'] = prob
            yield temp

                
class MyModel(LdaModel):
    """
    Subclass of gensim.models.LdaModel.
    """
    def get_coherence(self, corpus):
        """
        Returns the average coherence measure for the given model.

        Args:
            corpus (MyCorpus): A corpus on which the model is computed. 

        Returns:
            float: the average coherence measure for the given model.
        """
        top_topics = self.top_topics(corpus)
        return sum([t[1] for t in top_topics]) / len(top_topics)
    
    def get_top_tokens(self, corpus):
        """
        Returns a list of dictionaries that depict the most probable
        tokens for each topic.

        Args:
            corpus (MyCorpus): A corpus on which the model was computed.

        Returns:
            list: list of dicitionaries that depict the most probable 
            tokens fro each topic.
        """
        top_tokens = self.top_topics(corpus)
        return [ { key : value for value, key in t[0] } for t in top_tokens[:10] ]

    
    
        
def run_lda_models(corpus, dictionary, min_topics, max_topics, step = 1, **kwargs):
    """
    Computes a sequence of lda models for a given corpus and dictionary. It prints
    the coherence measure and number of topics to the screen. It writes out the
    model to disk.

    Args:
        corpus (MyModel): A stream of document vectors or sparse matrix of shape (num_documents, num_terms).
        dictionary (dict): a mapping that assigns id to unique tokens from the corpus.
        min_topics (int): the smallest number of topics to compute.
        max_topics (int): the highest number of topics to compute.
        step (int, optional): the size of the break inbetween computed models. Defaults to 1.
    """
    name = input("Please provide the name of the model\n")
    temp = dictionary[0]
    id2word = dictionary.id2token
    if not os.path.exists('models'):
        os.mkdir('models')
    if not os.path.exists('png'):
        os.mkdir('png')
    for num_topic in range(min_topics, max_topics+1, step):
        model = MyModel( corpus = corpus,
                         id2word=id2word,
                         alpha = 'asymmetric',
                         eta = 'auto',
                         iterations = 500,
                         passes = 20,
                         eval_every=None,
                         num_topics=num_topic,
                         random_state=1044,
                         per_word_topics=True)
        temp_dict = {}
        temp_dict['name'] = name
        temp_dict['num_topics'] =  num_topic
        temp_dict['coherence'] = model.get_coherence(corpus = corpus)
        path_name = os.path.join('models', name + '-' + str(num_topic))
        model.save(path_name) 
        print(temp_dict)

In [343]:
#@title Basics
data = []
for line in open('Reviews.jl', 'r'):
  data.append(json.loads(line))
reviews = data[0]['reviews']
reviews_corrected = []
for i in reviews.items():
    reviews_corrected.append(i[1])
    
positive_reviews = []
negative_reviews = []
for i in reviews_corrected:
  if i['voted_up'] == True:
    positive_reviews.append(i)
  else:
    negative_reviews.append(i)

In [344]:
#@title Cleaning reviews
import re
from nltk.tokenize import word_tokenize

Reviews_Content = []
for i in reviews_corrected:
  temp_dict = {}
  temp_dict['content'] = i['review']
  Reviews_Content.append(temp_dict)

Negative_Reviews_Content = []
for i in negative_reviews:
  temp_dict = {}
  temp_dict['content'] = i['review']
  Negative_Reviews_Content.append(temp_dict)


In [ ]:
#@title Relevant tokens

#Cleaning and tokenizing into sentences
List_of_tokens = []
for d in Reviews_Content:
 for i in d.values():
  i = re.sub(r'[^\w\s,!.\'\"]', '', i)
  i = i.replace('\n', '')
  tokens = sent_tokenize(i.lower())
  temp_list = []
  for token in tokens:
      token = re.sub(r'\d', '', token)
      token = re.sub(r'[^\w\s]', '', token)
      temp_list.append(token)
  List_of_tokens.append(temp_list)
List_of_tokens

key_words = ["trans", "transgender", 'transsexual', "transvestite", "transphobic", "transphobia", "lgbt", "lgbtq", "rights", "ideology", "jk", "rowling", "Sirona", "Ryan", "boycott"]

#Some people write "j k" instead of "jk" when refering to Rowling, so we have to first fix that:
for i in range(len(List_of_tokens)):
    sub_list = List_of_tokens[i]
    for j in range(len(sub_list)):
        s = sub_list[j]
        if "j" in s and "k" in s and s.index("j") < s.index("k"):
            s = s.replace(" ", "") if " j k " in s else s
            sub_list[j] = s

#For each review I take only the tokens (sentences) that regard trans rights topic
#As You can see, I am using a re function to make sure only the exact words are included,
#The reason for that is that there is a lot of noise (eg. words like "transparent") otherwise.

#Targetting relevant sentences
Relevant_sentences = []
for review in List_of_tokens:
    temp_list = []
    for sentence in review:
        if any(re.search(r"\b{}\b".format(word), sentence) for word in key_words):
            temp_list.append(sentence)
            Relevant_sentences.append(temp_list)

#Removing the duplicates
Relevant_sentences = list(set(map(tuple, Relevant_sentences)))
Relevant_sentences = [list(l) for l in Relevant_sentences]
Relevant_sentences

In [ ]:
#@title Dealing with noise

#Deleting stopwords and other noise in the data
stop_words = set(stopwords.words('english'))
stop_words.add('game')
stop_words.add('im')
#top_words.add('trans')
#top_words.add('jk')
#stop_words.add('rowling')
#stop_words.add('rowle')
#stop_words.add('transphobic')


filtered_list = []
for i in Relevant_sentences:
  temp_list = []
  for string in i:
    # Tokenize the string into words so I can remove stopwords
    words = string.split()
    # Filter out the stopwords from the words
    filtered_words = [word for word in words if word not in stop_words]
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    # Update the value in the dictionary with the filtered text
    temp_list.append(filtered_text)
  filtered_list.append(temp_list)
#Note1: I have to join them back into a string because the MyCorpus command tokenizes them again
#Note2: I join them in a way to preserve the original structure.
#As such most of the results will be composed of list containing one element
#Because most reviews contained one sentence regarding the revelant topic
#However if there were more sentences, the list will have more elements

#Writing them into a dictionary
#Now if there were more sentences, they will be connected together
review_dict = {}
for i, review in enumerate(filtered_list):
  for content in review:
    review_dict[i] = {"id": i, "content": content}

#Saving the dictionary into a file
with open("Relevant_reviews.jl", "w") as f:
    for review in review_dict.values():
        f.write(json.dumps(review) + "\n")
review_dict

As we can see, 1539 reviews regard the topic of interest. However, there is still a considerable ammount of noise in the data, we will see whether LDA model will allow to better distinguish between noise and relevant topic.

In [ ]:
#@title Creating model

corpus = MyCorpus(path = 'Relevant_reviews.jl')

## Create the dictionary
dictionary = Dictionary(corpus.get_tokens())

## Filter out words that occur in less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below= 20, no_above=0.5)

## Add the dictionary to the corpus
corpus.set_dictionary(dictionary)

run_lda_models(corpus = corpus, dictionary = dictionary, min_topics=2, max_topics=12)

In [ ]:
#@title Visualizing model

## Read in the module. It requires providing
## the name of the model we want to load.
model_name = input('Provide the name of the model you would like to load:\n')
model_path = os.path.join('models', model_name)
model = LdaModel.load(model_path)

## Print out and write the figures with the most 
## probable tokens in each topic.
list_top_tokens = model.get_top_tokens(corpus)
for i in range(len(list_top_tokens)):
    plt.barh(list(list_top_tokens[i].keys()), list(list_top_tokens[i].values()), align = 'center')
    plt.xlim(0,.03)
    plt.gca().invert_yaxis()
    plt.title('Topic' + ' ' + str(i + 1))
    plt.xlabel('Probability')
    plt.savefig('png/' + 'topic' + str(i + 1))
    plt.show()

The results came out far better than expected. Since only sentences that include topic of interest were selected a high degree of intertwining was to be expected. Turns out that LDA still performed very well:

**Topic 1 & 3** represent World of Harry Potter and are composed mostly of sentences that praise the setting of the game and the world that Rowling has written

**Topic 2** regards the issue of trans rights and game's boycott

Topic 2 selects reviews that represent the topic of interest and eliminates even more noise that was still present in the data.

Could we possibly use sentiment scores to be even more certain that a given review represents our topic of interest? Let's see if there exists an optimal value of a review's sentiment that could function as a cut-off for a review to be classified as belonging to the topic of interest.

(To proceed You have to choose a model with three topics)

In [ ]:
#@title How do the documents in those topics look like?

# Creating a list of tuples containing the document index and its topic distribution
doc_topic_dist = model.get_document_topics(corpus)
doc_topic_tuples = list(enumerate(doc_topic_dist))

# Grouping the documents by their most likely topic
grouped_docs = {}
for doc_id, topics in doc_topic_tuples:
    topic = max(topics, key=lambda x: x[1])[0]
    if topic not in grouped_docs:
        grouped_docs[topic] = []
    grouped_docs[topic].append((doc_id, topics[1]))

# Obtaining top 10 documents for each topic
top_docs_per_topic = {}
for topic, docs in grouped_docs.items():
    topic_docs = sorted(docs, key=lambda x: x[1], reverse=True)[:10]
    top_docs_per_topic[topic] = [x[0] for x in topic_docs]


Topic_1 = {id: review_dict[id]['content'] for id in top_docs_per_topic[0] if id in review_dict}
Topic_2 = {id: review_dict[id]['content'] for id in top_docs_per_topic[1] if id in review_dict}
Topic_3 = {id: review_dict[id]['content'] for id in top_docs_per_topic[2] if id in review_dict}

print("Topic 1:")
for key, value in Topic_1.items():
    print(value)
print('\n')

print("Topic 2:")
for key, value in Topic_2.items():
    print(value)
print('\n')

print("Topic 3:")
for key, value in Topic_3.items():
    print(value)
print('\n')

In [ ]:
#@title Sentiments for all reviews

import numpy as np

Reviews_scores = []
for item in Relevant_sentences:
    temp_dict = {}
    temp_dict['Content'] = item
    Reviews_scores.append(temp_dict)

sentiment_scores_all = []
for review in Relevant_sentences:
  strings_sentiment = []
  for string in review:
        score = (vader.polarity_scores(string))
        sentiment = score['compound'] * (1 - score['neu'])
        strings_sentiment.append(sentiment)
  sentiment_scores_all.append(strings_sentiment)

sentiment_list = []
for i in sentiment_scores_all:
  if len(i) > 1:
    mean_score = np.mean(i)
  else:
    mean_score = i[0]
  sentiment_list.append(mean_score)

for i in range(len(Reviews_scores)):
    Reviews_scores[i]['Sentiment'] = sentiment_list[i]
Reviews_scores

In [ ]:
#@title Is the sentiment correlated with Topic relatedness?
import seaborn as sns
import matplotlib.pyplot as plt

Topic_relatedness = []
for item, value in doc_topic_tuples:
      Topic_relatedness.append(value[1][1].item()) #the .item() is needed as it was a numpy float and not a normal float

#Scaling values to range from 0 to 1
Topic_relatedness_scaled = []
for i in Topic_relatedness:
  i = (i + 1) /2
  Topic_relatedness_scaled.append(i)

for i in range(len(Reviews_scores)):
  Reviews_scores[i]['Topic2_Relatedness'] = Topic_relatedness_scaled[i]
Reviews_scores

# Extract the values to be plotted from the dictionary
x = Topic_relatedness_scaled
y = sentiment_list

# Create a scatterplot using Seaborn
sns.scatterplot(x=x, y=y)

# Set the title and labels for the plot
plt.title("Scatter Plot of Data")
plt.xlabel("How much is a given review related to Topic 2")
plt.ylabel("Sentiment of a given review")

# Show the plot
plt.show()

NOTE: I have just realized that LDA displays Topics in a random order, as such the graph above may not represent the relevant topic. This is not however a problem for further analysis and I discuss the relevant graph below anyways

The results are not very informative. I hoped the chance of a review belonging to topic 2 would have a higher correlation with the sentiment of a language used. Perhaps it is, but the correlation can not be so easily found because the stances are so polarized: one reviews are using positive language and others negative language when referring to the same topic of trans inclusion.

Now we have raviews that we can be quite certain that in fact represent our topic of interest. However, there is still a need to distinguish review's stance on the matter.

The relationship between language sentiment and review's topic is complicated by opposing stances. Reviews like "I hate lgbt ideology" and "I hate transphobia" will have an identical sentiment value AND (nearly) identical chance of being detected as belonging to Topic 2, but in fact represent completly opposing stances

How to determine what is the review's stance on the matter?

There is a solution! Bigram and trigram modeling allow for taking under consideration the co-occurrence of words in pairs and triplets.


In [ ]:
#@title Collocation analysis
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

#Some reviews contain more than one sentence, so we need to first join them before tokenizing
word_tokens = []
for i in filtered_list:
  temp_list = []
  if len(i) > 1:
      temp_list.append(' '.join(i))
  else:
      word_tokens.append(i)
  if len(temp_list) > 0:
    word_tokens.append(temp_list)
    

#Now tokenize each review
word_tokens_proper = []
for item in word_tokens:
  for i in item:
    word_tokens_proper.append(word_tokenize(i))
word_tokens_proper

#Now let's connect "jk" and "rowling"
for lst in word_tokens_proper:
    for i in range(len(lst)-1):
        if lst[i] == "jk" and lst[i+1] == "rowling":
            lst[i] = "jkrowling"
            lst.pop(i+1)
            break
word_tokens_proper

anchor_words = ["trans", "transgender", 'transsexual', "transvestite", "transphobic", "transphobia", "lgbt", "lgbtq", "rights", "ideology", "jk", "rowling", "sirona", "ryan", "boycott", "woke", "sjw", "artist"]
tokens = [token for word_tokens_proper in word_tokens_proper for token in word_tokens_proper]

finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(4) # only consider bigrams that appear at least 5 times
finder.apply_word_filter(lambda w: len(w) < 3) # ignore bigrams containing words with less than 3 characters

def anchor_filter(w1, w2):
    return w1 in anchor_words or w2 in anchor_words

finder.apply_ngram_filter(anchor_filter)

finder.apply_ngram_filter(anchor_filter)

measures = BigramAssocMeasures()
scored_bigrams = finder.score_ngrams(measures.chi_sq)
top_bigrams = finder.nbest(measures.chi_sq, 30)
scored_bigrams

#ID's of bigrams representing different stances
negative = [4, 7, 10, 19, 33, 43, 46, 55, 56, 83, 94]
positive = [69, 73, 103, 122, 126, 136, 146]

selected_bigrams = []
for i in negative:
    selected_bigrams.append(scored_bigrams[i])

In [ ]:
#@title Bigram modeling

#Dictionary and corpus
dictionary = Dictionary([tokens for tokens in word_tokens_proper])
corpus = [dictionary.doc2bow(tokens) for tokens in word_tokens_proper]

#LDA
lda_model = LdaModel(corpus, num_topics= 2, id2word=dictionary, passes=10)

In [ ]:
#@title Words in topics

# Print the top 5 words for each topic
for topic_id in range(lda_model.num_topics):
    top_words = model.show_topic(topic_id, topn=10)
    print("Topic {}: {}".format(topic_id, top_words))

In [ ]:
#@title Topic visualization

vis_filtered = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_filtered)

In [ ]:
#@title End result, model validation

# Creating a list of tuples containing the document index and its topic distribution
doc_topic_dist = lda_model.get_document_topics(corpus)
doc_topic_tuples = list(enumerate(doc_topic_dist))

# Grouping the documents by their most likely topic
grouped_docs = {}
for doc_id, topics in doc_topic_tuples:
    topic = max(topics, key=lambda x: x[1])[0]
    if topic not in grouped_docs:
        grouped_docs[topic] = []
    grouped_docs[topic].append((doc_id, topics[0]))

# Obtaining top 10 documents for each topic
top_docs_per_topic = {}
for topic, docs in grouped_docs.items():
    topic_docs = sorted(docs, key=lambda x: x[1], reverse=True)[:10]
    top_docs_per_topic[topic] = [x[0] for x in topic_docs]


Topic_1 = {id: review_dict[id]['content'] for id in top_docs_per_topic[0] if id in review_dict}
Topic_2 = {id: review_dict[id]['content'] for id in top_docs_per_topic[1] if id in review_dict}

print("Topic 1:")
for key, value in Topic_1.items():
    print(value)
print('\n')

print("Topic 2:")
for key, value in Topic_2.items():
    print(value)
print('\n')

**In conclusion**

To determine whether it is possible to predict review's topic, an LDA model was created and followed with sentiment analysis. Next, a bigram model based on collocation analysis was carried out. End results allow for differentiation between reviewer's stance on the topic of trans inclusion and boycott.